In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import json
import numpy as np
import pandas as pd
import keras
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [4]:
x_train = np.load('/content/drive/My Drive/data.npy')
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
y_train = np.load('/content/drive/My Drive/lable.npy')
y_train = keras.utils.to_categorical(y_train, num_classes = 2)

In [5]:
print(x_train.shape)
print(y_train.shape)

(1309, 224, 224, 3)
(1309, 2)


In [6]:

x_train = x_train.astype('float32')
x_train /= 255

In [7]:
densenet = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
model = Sequential()
model.add(densenet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2050      
Total params: 7,039,554
Trainable params: 6,955,906
Non-trainable params: 83,648
_________________________________________________________________
None


In [8]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['AUC']
)

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=20,
    validation_split=0.1
)


Epoch 1/20
19/19 [==============================] - 826s 43s/step - loss: 0.5846 - auc: 0.8507 - val_loss: 1.4454 - val_auc: 0.7737
Epoch 2/20
19/19 [==============================] - 814s 43s/step - loss: 0.2743 - auc: 0.9532 - val_loss: 1.2301 - val_auc: 0.7951
Epoch 3/20
19/19 [==============================] - 814s 43s/step - loss: 0.1986 - auc: 0.9752 - val_loss: 3.8677 - val_auc: 0.7786
Epoch 4/20
19/19 [==============================] - 821s 43s/step - loss: 0.1249 - auc: 0.9897 - val_loss: 2.9610 - val_auc: 0.4979
Epoch 5/20
19/19 [==============================] - 820s 43s/step - loss: 0.1197 - auc: 0.9911 - val_loss: 1.7539 - val_auc: 0.8181
Epoch 6/20
19/19 [==============================] - 823s 43s/step - loss: 0.1057 - auc: 0.9930 - val_loss: 1.9639 - val_auc: 0.8245
Epoch 7/20
19/19 [==============================] - 827s 44s/step - loss: 0.1146 - auc: 0.9915 - val_loss: 1.7127 - val_auc: 0.7456
Epoch 8/20
19/19 [==============================] - 823s 43s/step - loss: 0.

In [9]:

model.save_weights("densenet_weights.h5")

In [ ]:
!ls

densenet_weights.h5  drive  sample_data


In [ ]:
from google.colab import files
files.download('densenet_weights.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
x_test = np.load('/content/drive/My Drive/test_images.npy')
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
y_test = np.load('/content/drive/My Drive/test_labels.npy')
x_test = x_test.astype('float32')
x_test /= 255
x_test.shape

(98, 224, 224, 3)

In [11]:
result = model.predict_classes(x_test)
print(result)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [14]:
sum1 = 0
sum2 = 9
for i in range(len(result)):
  if(result[i] == y_test[i]):
    sum1 += 1
  else:
    sum2 += 1
print(sum1)
print(sum2)
sum1/len(result)


68
39


0.6938775510204082